![JohnSnowLabs](https://sparknlp.org/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp/blob/master/examples/python/transformers/openvino/HuggingFace_OpenVINO_in_Spark_NLP_snowflake.ipynb)

# Import OpenVINO snowflake  models from HuggingFace 🤗 into Spark NLP 🚀

This notebook provides a detailed walkthrough on optimizing and exporting BGE models from HuggingFace for use in Spark NLP, leveraging the various tools provided in the [Intel OpenVINO toolkit](https://www.intel.com/content/www/us/en/developer/tools/openvino-toolkit/overview.html) ecosystem.

Let's keep in mind a few things before we start 😊

- OpenVINO support was introduced in  `Spark NLP 5.4.0`, enabling high performance inference for models. Please make sure you have upgraded to the latest Spark NLP release.
- You can import models for snowflake  from snowflake  and they have to be in `Fill Mask` category.

## 1. Export and Save the HuggingFace model

- Let's install `transformers` and `openvino` packages with other dependencies. You don't need `openvino` to be installed for Spark NLP, however, we need it to load and save models from HuggingFace.
- We lock `transformers` on version `4.48.3`. This doesn't mean it won't work with the future releases, but we wanted you to know which versions have been tested successfully.

In [ ]:
!pip install -q --upgrade transformers==4.48.3 optimum[openvino]

[Optimum Intel](https://github.com/huggingface/optimum-intel?tab=readme-ov-file#openvino) is the interface between the Transformers library and the various model optimization and acceleration tools provided by Intel. HuggingFace models loaded with optimum-intel are automatically optimized for OpenVINO, while being compatible with the Transformers API.
- To load a HuggingFace model directly for inference/export, just replace the `AutoModelForXxx` class with the corresponding `OVModelForXxx` class. We can use this to import and export OpenVINO models with `from_pretrained` and `save_pretrained`.
- By setting `export=True`, the source model is converted to OpenVINO IR format on the fly.
- We'll use [Snowflake/snowflake-arctic-embed-m](https://huggingface.co/Snowflake/snowflake-arctic-embed-m) model from HuggingFace as an example and load it as a `OVModelForFeatureExtraction`, representing an OpenVINO model.
- In addition to the OVModelForFeatureExtraction model, we also need to save the `AutoTokenizer`. This is the same for every model, these are assets (saved in `/assets`) needed for tokenization inside Spark NLP.

In [2]:
from optimum.intel import OVModelForFeatureExtraction
from transformers import AutoTokenizer

MODEL_NAME = "Snowflake/snowflake-arctic-embed-m"
EXPORT_PATH = f"ov_models/{MODEL_NAME}"

ov_model = OVModelForFeatureExtraction.from_pretrained(MODEL_NAME, export=True, trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

ov_model.save_pretrained(EXPORT_PATH)
tokenizer.save_pretrained(EXPORT_PATH)

No CUDA runtime is found, using CUDA_HOME='/usr/local/cuda'
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/738 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of BertModel were not initialized from the model checkpoint at Snowflake/snowflake-arctic-embed-m and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/1.38k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


('ov_models/Snowflake/snowflake-arctic-embed-m/tokenizer_config.json',
 'ov_models/Snowflake/snowflake-arctic-embed-m/special_tokens_map.json',
 'ov_models/Snowflake/snowflake-arctic-embed-m/vocab.txt',
 'ov_models/Snowflake/snowflake-arctic-embed-m/added_tokens.json',
 'ov_models/Snowflake/snowflake-arctic-embed-m/tokenizer.json')

Let's have a look inside these two directories and see what we are dealing with:

In [3]:
!ls -l {EXPORT_PATH}

total 426660
-rw-r--r-- 1 root root       675 Jun 24 12:18 config.json
-rw-r--r-- 1 root root 435570848 Jun 24 12:18 openvino_model.bin
-rw-r--r-- 1 root root    363466 Jun 24 12:18 openvino_model.xml
-rw-r--r-- 1 root root       695 Jun 24 12:18 special_tokens_map.json
-rw-r--r-- 1 root root      1411 Jun 24 12:18 tokenizer_config.json
-rw-r--r-- 1 root root    711649 Jun 24 12:18 tokenizer.json
-rw-r--r-- 1 root root    231508 Jun 24 12:18 vocab.txt


We need to move `vocab.txt` from the tokenizer into an `assets` folder, which is where Spark NLP expects to find it.


In [4]:
!mkdir {EXPORT_PATH}/assets && mv {EXPORT_PATH}/vocab.txt {EXPORT_PATH}/assets/

In [5]:
!ls -l {EXPORT_PATH}

total 426436
drwxr-xr-x 2 root root      4096 Jun 24 12:18 assets
-rw-r--r-- 1 root root       675 Jun 24 12:18 config.json
-rw-r--r-- 1 root root 435570848 Jun 24 12:18 openvino_model.bin
-rw-r--r-- 1 root root    363466 Jun 24 12:18 openvino_model.xml
-rw-r--r-- 1 root root       695 Jun 24 12:18 special_tokens_map.json
-rw-r--r-- 1 root root      1411 Jun 24 12:18 tokenizer_config.json
-rw-r--r-- 1 root root    711649 Jun 24 12:18 tokenizer.json


Voila! We have our `vocab.txt` inside assets directory

## Import and Save snowflake in Spark NLP


- Install and set up Spark NLP in Google Colab
- This example uses specific versions of `pyspark` and `spark-nlp` that have been tested with the transformer model to ensure everything runs smoothly.

In [6]:
!pip install -q pyspark==3.5.5  spark-nlp==5.5.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.2/317.2 MB 5.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 635.7/635.7 kB 34.2 MB/s eta 0:00:00


Let's start Spark with Spark NLP included via our simple `start()` function

In [7]:
import sparknlp

spark = sparknlp.start()

print("Spark NLP version: ", sparknlp.version())
print("Apache Spark version: ", spark.version)

Spark NLP version:  5.5.3
Apache Spark version:  3.5.5


- Let's use `loadSavedModel` functon in `SnowFlakeEmbeddings` which allows us to load the Openvino model
- Most params will be set automatically. They can also be set later after loading the model in `SnowFlakeEmbeddings` during runtime, so don't worry about setting them now
- `loadSavedModel` accepts two params, first is the path to the exported model. The second is the SparkSession that is `spark` variable we previously started via `sparknlp.start()`
- `setStorageRef` is very important. When you are training a task like NER or any Text Classification, we use this reference to bound the trained model to this specific embeddings so you won't load a different embeddings by mistake and see terrible results 😊
- It's up to you what you put in `setStorageRef` but it cannot be changed later on. We usually use the name of the model to be clear, but you can get creative if you want!
- The `dimension` param is is purely cosmetic and won't change anything. It's mostly for you to know later via `.getDimension` what is the dimension of your model. So set this accordingly.
- NOTE: `loadSavedModel` accepts local paths in addition to distributed file systems such as `HDFS`, `S3`, `DBFS`, etc. This feature was introduced in Spark NLP 4.2.2 release. Keep in mind the best and recommended way to move/share/reuse Spark NLP models is to use `write.save` so you can use `.load()` from any file systems natively.st and recommended way to move/share/reuse Spark NLP models is to use `write.save` so you can use `.load()` from any file systems natively.


In [8]:
from sparknlp.annotator import SnowFlakeEmbeddings

snowflake = SnowFlakeEmbeddings.loadSavedModel(f"{EXPORT_PATH}", spark)\
    .setInputCols(["document"])\
    .setOutputCol("snowflake")\
    .setCaseSensitive(True)\
    .setDimension(768)

- Let's save it on disk so it is easier to be moved around and also be used later via `.load` function

In [9]:
snowflake.write().overwrite().save(f"{MODEL_NAME}_spark_nlp")

Let's clean up stuff we don't need anymore

In [10]:
!rm -rf {EXPORT_PATH}

Awesome  😎 !

This is your Openvino snowflake model from HuggingFace 🤗  loaded and saved by Spark NLP 🚀

In [11]:
! ls -l {MODEL_NAME}_spark_nlp

total 425796
drwxr-xr-x 3 root root      4096 Jun 24 12:20 fields
drwxr-xr-x 2 root root      4096 Jun 24 12:20 metadata
-rw-r--r-- 1 root root 436001117 Jun 24 12:20 snowFlake_openvino


Now let's see how we can use it on other machines, clusters, or any place you wish to use your new and shiny snowflake model 😊

In [12]:
from sparknlp.base import DocumentAssembler
from sparknlp.annotator import SnowFlakeEmbeddings
from pyspark.ml import Pipeline

document_assembler = DocumentAssembler() \
    .setInputCol("text") \
    .setOutputCol("document")

snowflake_loaded = SnowFlakeEmbeddings.load(f"{MODEL_NAME}_spark_nlp") \
    .setInputCols(["document"]) \
    .setOutputCol("snowflake")

pipeline = Pipeline(stages=[
    document_assembler,
    snowflake_loaded
])

data = spark.createDataFrame([
    ["William Henry Gates III (born October 28, 1955) is an American business magnate, software developer, investor, and philanthropist."]
]).toDF("text")

model = pipeline.fit(data)
result = model.transform(data)

result.selectExpr("explode(snowflake.embeddings) as embeddings").show()

+--------------------+
|          embeddings|
+--------------------+
|[-0.60515916, 0.5...|
+--------------------+



That's it! You can now go wild and use hundreds of snowflake models from HuggingFace 🤗 in Spark NLP 🚀
